In [9]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
DataCleaner 完整版（仅移除变量类型判断与二分类清洗）
双击即可运行，参数已在底部写死
"""
import pandas as pd
import re
import logging
import numpy as np
import warnings
from pathlib import Path
from typing import Dict, List, Any
from openpyxl import load_workbook

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO, format="%(levelname)s | %(message)s")
logger = logging.getLogger("data_cleaner")

# ==================== 正则表达式模式 ====================
NUM_PATTERN = re.compile(r"-?\d+(?:\.\d+)?")
MISSING_PATTERN = re.compile(r".*missing.*form.*|.*incomplete.*workbook.*", re.IGNORECASE)

class DataCleaner:
    def __init__(self):
        pass

    def extract_number(self, text: str) -> Any:
        if pd.isna(text):
            return np.nan
        text_str = str(text).strip()
        if MISSING_PATTERN.match(text_str):
            return np.nan
        match = NUM_PATTERN.search(text_str)
        if match:
            num_str = match.group(0)
            try:
                return float(num_str) if '.' in num_str else int(num_str)
            except:
                return np.nan
        if not text_str or text_str.isspace():
            return np.nan
        return text_str

    def batch_extract_numbers(self, df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
        logger.info(f"批量正则提取数字，共{len(columns)}列...")
        total_cols = len(columns)
        for i, col in enumerate(columns, 1):
            try:
                df[col] = df[col].apply(self.extract_number)
                if i % 20 == 0 or i == total_cols:
                    logger.info(f"进度: {i}/{total_cols} 列 ({i/total_cols*100:.1f}%)")
            except Exception as e:
                logger.warning(f"处理列 {col} 时出错: {e}")
                continue
        return df

    # ---------- 已删除 detect_variable_types 和 clean_binary_variable ----------

    def remove_high_missing_ids(self, df: pd.DataFrame, id_col: str, threshold: float = 0.7) -> pd.DataFrame:
        if id_col not in df.columns:
            logger.warning(f"ID列 {id_col} 不存在，跳过ID删除步骤")
            return df
        id_missing_rate = df.groupby(id_col).apply(
            lambda x: x.drop(columns=[id_col]).isna().mean().mean()
        )
        valid_ids = id_missing_rate[id_missing_rate <= threshold].index.tolist()
        filtered_df = df[df[id_col].isin(valid_ids)].copy()
        removed_count = len(df) - len(filtered_df)
        if removed_count > 0:
            logger.info(f"删除缺失率 > {threshold*100}% 的ID: {removed_count} 行被删除")
        return filtered_df

    def clean_excel_inplace(self,
                           in_file: str | Path,
                           sheet_name: str | int = 0,
                           id_col: str = None,
                           missing_threshold: float=0.7, 
                           backup: bool = False) -> None:
        in_file = Path(in_file)
        if not in_file.exists():
            raise FileNotFoundError(in_file)
        if backup:
            backup_file = in_file.with_name(f"{in_file.stem}_backup_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.xlsx")
            import shutil
            shutil.copy2(in_file, backup_file)
            logger.info(f"已备份原文件到: {backup_file}")

        logger.info(f"读取文件: {in_file}, Sheet: {sheet_name}")
        try:
            df = pd.read_excel(in_file, sheet_name=sheet_name)
        except Exception as e:
            logger.error(f"读取Excel文件失败: {e}")
            raise

        original_rows = df.shape[0]
        logger.info(f"原始数据: {original_rows} 行 × {df.shape[1]} 列")

        if isinstance(sheet_name, int):
            with pd.ExcelFile(in_file) as xls:
                actual_sheet_name = xls.sheet_names[sheet_name]
        else:
            actual_sheet_name = sheet_name

        if id_col is None:
            id_col = df.columns[0]
            logger.info(f"使用第一列作为ID列: {id_col}")
        elif id_col not in df.columns:
            logger.warning(f"指定的ID列 {id_col} 不存在，使用第一列 {df.columns[0]} 作为ID列")
            id_col = df.columns[0]

        logger.info("第一步：删除高缺失率ID...")
        df = self.remove_high_missing_ids(df, id_col, missing_threshold)
        logger.info(f"删除后数据: {df.shape[0]} 行 × {df.shape[1]} 列")

        logger.info("第二步：批量正则提取数字...")
        id_data = df[id_col].copy()
        other_cols = [col for col in df.columns if col != id_col]
        df = self.batch_extract_numbers(df, other_cols)
        df[id_col] = id_data
        # ---------- 已删除变量类型判断与二分类清洗步骤 ----------

        logger.info(f"正在覆盖原sheet: {actual_sheet_name}...")
        with pd.ExcelWriter(in_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            df.to_excel(writer, sheet_name=actual_sheet_name, index=False)

        logger.info(f"✅ 清洗完成！已覆盖原文件的 {actual_sheet_name} sheet")
        self._print_summary(df, original_rows)

    def _print_summary(self, df: pd.DataFrame, original_rows: int):
        print("\n" + "="*60)
        print("数据清洗摘要")
        print("="*60)
        total_cells = df.shape[0] * df.shape[1]
        missing_cells = df.isna().sum().sum()
        missing_percent = missing_cells / total_cells * 100
        print(f"缺失统计:")
        print(f"  总单元格: {total_cells:,}")
        print(f"  缺失单元格: {missing_cells:,}")
        print(f"  缺失率: {missing_percent:.2f}%")
        removed_rows = original_rows - df.shape[0]
        print(f"行数变化:")
        print(f"  原始行数: {original_rows}")
        print(f"  清洗后行数: {df.shape[0]}")
        print(f"  删除行数: {removed_rows}")
        print(f"数据维度: {df.shape[0]} 行 × {df.shape[1]} 列")


# ==================== 一键执行 ====================
if __name__ == "__main__":
    INPUT_FILE = r"C:\Users\DXW\Desktop\半月板手术_JSW_OUT.xlsx"
    SHEET_NAME = "merged"          # 可改为任意 sheet 名或索引
    ID_COL     = None           # None 表示用第一列当 ID
    THRESHOLD  = 0.5            # 缺失率阈值
    BACKUP     = False          # 是否备份原文件

    cleaner = DataCleaner()
    try:
        cleaner.clean_excel_inplace(
            in_file=INPUT_FILE,
            sheet_name=SHEET_NAME,
            id_col=ID_COL,
            missing_threshold=THRESHOLD,
            backup=BACKUP
        )
        print(f"\n✅ 清洗完成！原文件 {INPUT_FILE} 的 {SHEET_NAME} sheet 已被更新")
    except Exception as e:
        logger.error(f"清洗失败: {e}")
        import traceback
        traceback.print_exc()

INFO | 读取文件: C:\Users\DXW\Desktop\半月板手术_JSW_OUT.xlsx, Sheet: merged
INFO | 原始数据: 246 行 × 30 列
INFO | 使用第一列作为ID列: months
INFO | 第一步：删除高缺失率ID...
INFO | 删除缺失率 > 50.0% 的ID: 66 行被删除
INFO | 删除后数据: 180 行 × 30 列
INFO | 第二步：批量正则提取数字...
INFO | 批量正则提取数字，共29列...
INFO | 进度: 20/29 列 (69.0%)
INFO | 进度: 29/29 列 (100.0%)
INFO | 正在覆盖原sheet: merged...
INFO | ✅ 清洗完成！已覆盖原文件的 merged sheet



数据清洗摘要
缺失统计:
  总单元格: 5,400
  缺失单元格: 767
  缺失率: 14.20%
行数变化:
  原始行数: 246
  清洗后行数: 180
  删除行数: 66
数据维度: 180 行 × 30 列

✅ 清洗完成！原文件 C:\Users\DXW\Desktop\半月板手术_JSW_OUT.xlsx 的 merged sheet 已被更新
